Conexão com o Azure Data Lake

![Conexão com o Azure Data Lake](https://raw.githubusercontent.com/joaovilar/Python/main/Python_Jupyter/from_container_to_database.png)

In [23]:
#importando as bibliotecas
from azure.storage.blob import BlobServiceClient
from io import BytesIO
import pandas as pd

In [24]:

# Chaves de acesso e URL do serviço
account_key = "SuaChaveDeAcesso"
account_name = "NomeDaConta"
container_name = "NomeDoContainer"

# Cria o cliente de serviço
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)


Leitura dos arquivos csv no container

In [25]:

def ler_arquivo_csv(account_key, account_name, container_name, blob_name, delimitador=';'):
    # Cria o cliente de serviço
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

    # Obtém referência para o blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    # Lê o conteúdo do blob (arquivo CSV) em um DataFrame
    blob_content = blob_client.download_blob().readall()
    # Especifique o delimitador usando o parâmetro sep
    df = pd.read_csv(BytesIO(blob_content), sep=delimitador)

    return df

# Exemplo de uso
df_stores = ler_arquivo_csv(account_key, account_name, container_name, "Excel/Stores.csv")
df_product = ler_arquivo_csv(account_key, account_name, container_name, "Excel/Product.csv")
df_channel = ler_arquivo_csv(account_key, account_name, container_name, "Excel/Channel.csv")
df_product_category = ler_arquivo_csv(account_key, account_name, container_name, "Excel/ProductCategory.csv")
df_geography = ler_arquivo_csv(account_key, account_name, container_name, "Excel/Geography.csv")


In [26]:
#tratando a coluna com informações nulas
df_stores['CloseReason'] = df_stores['CloseReason'].fillna('Not informed')
df_stores.head()

,StoreKey,GeographyKey,StoreType,StoreName,Status,CloseReason,EmployeeCount,SellingAreaSize
0,1,693,Store,Contoso Seattle No.1 Store,On,Not informed,17.0,462
1,2,693,Store,Contoso Seattle No.2 Store,On,Not informed,25.0,700
2,3,856,Store,Contoso Kennewick Store,On,Not informed,26.0,680
3,4,424,Store,Contoso Bellevue Store,On,Not informed,19.0,455
4,5,677,Store,Contoso Redmond Store,On,Not informed,33.0,560


In [27]:
df_product.head()

,ProductName,ProductDescription,Manufacturer,BrandName,ClassName,UnitCost,UnitPrice,ProductKey,ProductSubcategoryKey
0,Contoso Wireless Laser Mouse E50 Grey,Advanced 2.4 GHz cordless technology makes fre...,"Contoso, Ltd",Contoso,Economy,"10,69","20,96",873,22
1,Contoso Optical Wheel OEM PS/2 Mouse E60 Grey,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,879,22
2,Contoso Optical Wheel OEM PS/2 Mouse E60 Black,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,880,22
3,Contoso Optical Wheel OEM PS/2 Mouse E60 White,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,881,22
4,Contoso Optical Wheel OEM PS/2 Mouse E60 Silver,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,882,22


In [28]:
df_channel

,Channel,ChannelName
0,1,Store
1,2,Online
2,3,Catalog
3,4,Reseller


In [29]:
df_product_category

,ProductCategoryKey,ProductCategory
0,1,Audio
1,2,TV and Video
2,3,Computers
3,4,Cameras and camcorders
4,5,Cell phones
5,6,"Music, Movies and Audio Books"
6,7,Games and Toys
7,8,Home Appliances


In [30]:
df_geography.head()

,GeographyKey,GeographyType,ContinentName,RegionCountryName
0,424,City,North America,United States
1,430,City,North America,United States
2,431,City,North America,United States
3,432,City,North America,United States
4,433,City,North America,United States


Lista todos os blobs do azure storage

In [31]:
# Obtém referência para o contêiner
container_client = blob_service_client.get_container_client(container_name)

# Lista os blobs no contêiner
blobs = container_client.list_blobs()

# Itera sobre os blobs e faz alguma coisa (por exemplo, imprime o nome do blob)
for blob in blobs:
    print(f"Nome do Blob: {blob.name}")

# Lê o conteúdo do blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
conteudo_blob = blob_client.download_blob()


Nome do Blob: Carros.csv
Nome do Blob: Excel
Nome do Blob: Excel/Calendar.csv
Nome do Blob: Excel/Channel.csv
Nome do Blob: Excel/Geography.csv
Nome do Blob: Excel/Product.csv
Nome do Blob: Excel/ProductCategory.csv
Nome do Blob: Excel/ProductSubcategory.csv
Nome do Blob: Excel/Sales.csv
Nome do Blob: Excel/Stores.csv
Nome do Blob: raw


Conexão com o Azure SQL

In [32]:
#importando as bibliotecas
from sqlalchemy import create_engine
import pyodbc

In [33]:
# Configurações de conexão
server_name = "seu_servidor.database.windows.net"
database_name = "seu_banco_de_dados"
username = "seu_usuario"
password = "sua_senha"

In [34]:
def criar_conexao_azure_sql(server_name, database_name, username, password):
    # Define a string de conexão
    connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password};"

    # Cria a conexão
    conexao = pyodbc.connect(connection_string)

    return conexao

In [35]:

# Cria a string de conexão
connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"

# Cria uma engine de conexão usando SQLAlchemy
engine = create_engine(connection_string, echo=False)


In [36]:
# Inserir dados na tabela stg_stores
df_stores.to_sql(name="stg_stores", con=engine, if_exists="replace", index=False)
print('Tabela carregada')

Tabela carregada


In [37]:
# Inserir dados na tabela stg_Product
df_product.to_sql(name="stg_product", con=engine, if_exists="replace", index=False)
print('Tabela carregada')

Tabela carregada


In [38]:
# Inserir dados na tabela stg_channel
df_channel.to_sql(name="stg_channel", con=engine, if_exists="replace", index=False)
print('Tabela carregada')

Tabela carregada


In [39]:
# Inserir dados na tabela stg_geography
df_geography.to_sql(name="stg_geography", con=engine, if_exists="replace", index=False)
print('Tabela carregada')

Tabela carregada


In [40]:
# Inserir dados na tabela stg_product_category
df_product_category.to_sql(name="stg_product_category", con=engine, if_exists="replace", index=False)
print('Tabela carregada')

Tabela carregada


In [41]:
# Fechar a conexão
engine.dispose()